##Identifier le groupe

❌ Cette fonction idToGroup on la fournit l'id de l'étudiant à qui on veut faire la recommandation et le code du module ou il se trouve maintenant(dev) et cette fonction va retourner le num de groupe ou` il appartient cet etudiant



In [ ]:
#id=2698535
#module='EEE'
#presentation='2013J'

def idToGroup(id,module,presentation,grouping):
  dict1= dict()
  dict1= {'module': module,'presentation': presentation, 'group': int(grouping.loc[(grouping["id_student"] == id) & (grouping["code_module"]==module)&(grouping["code_presentation"]==presentation)]["grouping"].values), 'valuesLS':grouping.loc[(grouping["id_student"] == id) & (grouping["code_module"]==module)&(grouping["code_presentation"]==presentation), ["clickrate_active","clickrate_reflective", "clickrate_global", "clickrate_sensing", "clickrate_verbal", "clickrate_visual", "clickrate_sequential"]].values.tolist()[0]}
  return dict1

#dict1=idToGroup(id,module,presentation)


❌Ici j'ai enregistré dans un csv le groupement en eliminant id_student avec nettoyage à la fin

In [ ]:
#grouping_without_id = grouping.drop(["id_student","score"], axis=1)

In [ ]:
#grouping_without_id= grouping_without_id.drop_duplicates()

In [ ]:
#path = '/content/drive/MyDrive/Colab Notebooks/GroupingLS_withoutID_SumScoreEachStudent_SumClickrate.csv'
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#  grouping_without_id.to_csv(f)

❌ Cette fonction LSGroup(dict1) retourne à quel LS appartient un groupe

In [ ]:
def LSGroup(dict1,grouping_without_id):

  grouping1 = grouping_without_id.loc[(grouping_without_id['grouping']==dict1['group'])&(grouping_without_id['code_module']==dict1['module'])&(grouping_without_id['code_presentation']==dict1['presentation'])]
  
  min = float(grouping1['clickrate_active'].drop_duplicates())
  LS = 'clickrate_active'
  for x in ["clickrate_reflective", "clickrate_global", "clickrate_sensing", "clickrate_verbal", "clickrate_visual", "clickrate_sequential"]:

    if (min>float(grouping1[x].drop_duplicates())) :
      min = float(grouping1[x].drop_duplicates())
      LS = x

  return LS

In [ ]:
#LSGroup(dict1)

##Searching for the most active students in LS founded with module and presentation specified


❌Ici je vais automatiser le fait de rechercher dans quel cluster a mis le KMeans ceux les plus actifs en LS donné

In [ ]:
def clusterOfMostActive(groupingspecified,LS):
  cluster=2
  moyLS = groupingspecified[groupingspecified["cluster"]==2].describe(include= 'all').loc['mean'][LS]
  for i in range(2):
    if ((groupingspecified[groupingspecified["cluster"]==i].describe(include= 'all').loc['mean'][LS]) < moyLS):
      moyLS = groupingspecified[groupingspecified["cluster"]==i].describe(include= 'all').loc['mean'][LS]
      cluster = i
  return cluster

In [ ]:
#clusterOfMostActive()

In [ ]:
#mostactive=groupingspecified.loc[(groupingspecified['cluster']==clusterOfMostActive())].sort_values(LS)

In [ ]:
#mostactive = mostactive[["id_student", "clickrate_active", "clickrate_reflective", "clickrate_global", "clickrate_sensing", "clickrate_verbal", "clickrate_visual", "clickrate_sequential", "code_module", "code_presentation", "grouping", "score", "cluster"]]

In [ ]:
#mostactive=mostactive.drop_duplicates()

In [ ]:
#mostactive=mostactive.head(10)

In [ ]:
#mostactive

##bestactive in our module and presentation

❌ LENNA L MODULE oel presentation BCH NJIBOUHom ML PARTIE DEV (FINOU HOUA TAW) wala m dict idha deja jebnehom kbal

In [ ]:
#bestactive=mostactive

In [ ]:
#sse11 = []
#k_rng = range(1,5)
#for k in k_rng:
#   km = KMeans(n_clusters=k, n_init='auto')
#    km.fit(bestactive[["score"]])
#    sse11.append(km.inertia_)

In [ ]:
#plt.xlabel('K')
#plt.ylabel('Sum of squared error')
#plt.plot(k_rng,sse11)

In [ ]:
#km = KMeans(n_clusters=3)
#y_predicted = km.fit_predict(bestactive[["score"]])
#bestactive['cluster_score']=y_predicted

In [ ]:
def clusterOfBestActif(bestactive):
  cluster=2
  moyScore=bestactive[bestactive["cluster_score"]==2].describe(include= 'all').loc['mean']['score']
  for i in range(2):
    if bestactive[bestactive["cluster_score"]==i].describe(include= 'all').loc['mean']['score']>moyScore:
      moyLS=bestactive[bestactive["cluster_score"]==i].describe(include= 'all').loc['mean']['score']
      cluster=i
  return cluster

#clusterOfBestActif()

In [ ]:
#bestactive=bestactive.loc[(bestactive['cluster_score']==clusterOfBestActif())].sort_values('score', ascending=False)

In [ ]:
#bestactive = bestactive.head(10)

In [ ]:
#bestactive

##Similarity : Cosine

In [ ]:
#li = bestactive[["clickrate_reflective",	"clickrate_global",	"clickrate_sensing"	,"clickrate_verbal"	,"clickrate_visual","clickrate_sequential"]].values.tolist()

In [ ]:
#print(li)

In [ ]:
id=2698535
module='EEE'
presentation='2013J'

def TargetGroup(id,module,presentation,learningstylegrouping):
  from sklearn.cluster import KMeans
  from sklearn.preprocessing import MinMaxScaler
  from math import sqrt
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  from mpl_toolkits import mplot3d
  import sys
  import warnings
  if not sys.warnoptions:
      warnings.simplefilter("ignore")
  from google.colab import drive
  drive.mount('/content/drive')

  grouping = pd.read_csv("/content/drive/MyDrive/Colab Notebooks (1)/Grouping_"+learningstylegrouping+"_with_SumClickrates.csv")
  grouping_with_only_sumScore = pd.read_csv("/content/drive/MyDrive/Colab Notebooks (1)/Grouping_"+learningstylegrouping+"_with_only_SumScore.csv")

  dict1=idToGroup(id,module,presentation,grouping)

  grouping_without_id = grouping.drop(["id_student","score"], axis=1)
  grouping_without_id= grouping_without_id.drop_duplicates()

  LS=LSGroup(dict1,grouping_without_id)

  groupingspecified=grouping_with_only_sumScore[(grouping_with_only_sumScore["code_module"]==dict1['module'])&(grouping_with_only_sumScore["code_presentation"]==dict1['presentation'])]

  km = KMeans(n_clusters=3)
  y_predicted = km.fit_predict(groupingspecified[[LS]])
  groupingspecified['cluster']=y_predicted

  mostactive=groupingspecified.loc[(groupingspecified['cluster']==clusterOfMostActive(groupingspecified,LS))].sort_values(LS)
  mostactive = mostactive[["id_student", "clickrate_active", "clickrate_reflective", "clickrate_global", "clickrate_sensing", "clickrate_verbal", "clickrate_visual", "clickrate_sequential", "code_module", "code_presentation", "grouping", "score", "cluster"]]
  mostactive=mostactive.drop_duplicates()

  bestactive=mostactive
  km = KMeans(n_clusters=3)
  y_predicted = km.fit_predict(bestactive[["score"]])
  bestactive['cluster_score']=y_predicted

  bestactive=bestactive.loc[(bestactive['cluster_score']==clusterOfBestActif(bestactive))].sort_values('score', ascending=False)
  bestactive = bestactive.head(10)

  li = bestactive[["clickrate_reflective",	"clickrate_global",	"clickrate_sensing"	,"clickrate_verbal"	,"clickrate_visual","clickrate_sequential"]].values.tolist()

  sim_list=[]
  del dict1["valuesLS"][0]#car LS est clickrate_active qui est d'indice 0 dans cette liste
  x=dict1["valuesLS"]
  print(x)

  def square_rooted(x):
      return round(sqrt(sum([a*a for a in x])),3)


  def cosine_similarity(x,y):
      numerator = sum(a*b for a,b in zip(x,y))
      denominator = square_rooted(x)*square_rooted(y)
      return round(numerator/(float(denominator)+0.001),3)#pour eviter la division par zero, fe9nech biha lmara lokhra lfaza

  for i in range(len(bestactive.index)):
    sim = cosine_similarity(li[i],x)
    sim_list.append(sim)
  
  df = pd.DataFrame(list(zip(bestactive["id_student"],sim_list)),columns =["id_student",'similarity']).h
  df.sort_values(by=['similarity'],inplace=True)
  
  group = df
  return group

In [ ]:
idToGroup(id,module,presentation,grouping)

{'module': 'EEE',
 'presentation': '2013J',
 'group': 291,
 'valuesLS': [0.0354274917335852,
  6.113766484019752e-05,
  2.900692127454627e-06,
  1.428033047362278e-05,
  5.355123927608542e-06,
  0.0,
  4.462603273007119e-07]}

In [ ]:
TargetGroup(id,module,presentation,"clickrate_active")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-201-47d5352388ca>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groupingspecified['cluster']=y_predicted
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[6.113766484019752e-05, 2.900692127454627e-06, 1.428033047362278e-05, 5.355123927608542e-06, 0.0, 4.462603273007119e-07]


,id_student,similarity
1,1861793,0.000
2,587906,0.000
4,608243,0.000
7,1725109,0.000
3,610575,0.001
9,107339,0.001
5,445972,0.002
8,600945,0.002
0,246060,0.007
6,592961,0.045


In [ ]:
#from math import*

#sim_list=[]
#dict1 = idToGroup(id,module,presentation)
#print(dict1)
#del dict1["valuesLS"][0]#car LS est clickrate_active qui est d'indice 0 dans cette liste
#x=dict1["valuesLS"]
#print(x)

#def square_rooted(x):
 #   return round(sqrt(sum([a*a for a in x])),3)


#def cosine_similarity(x,y):
   # numerator = sum(a*b for a,b in zip(x,y))
  #  denominator = square_rooted(x)*square_rooted(y)
 #   return round(numerator/(float(denominator)+0.001),3)#pour eviter la division par zero, fe9nech biha lmara lokhra lfaza

#for i in range(len(bestactive.index)):
  # sim = cosine_similarity(li[i],x)
 #  sim_list.append(sim)

#print(sim_list)

In [ ]:
#df = pd.DataFrame(list(zip(bestactive["id_student"],sim_list)),columns =["id_student",'similarity'])

In [ ]:
#df.sort_values(by=['similarity'],inplace=True)

In [ ]:
#Target_Group = df.head(4)

In [ ]:
#Target_Group

✅ SAYE HEDHOM TARGET GROUP MTE3 L'ETUDIANT MTE3I ELLI MAWJOUD F GROUPE X MOU3AYEN avec similarity degree each target student ✅

